# 对GWAS结果的sumstats文件质控

## 一般我们下载的sumstats文件是可能不对的

这个时候就需要我们进行一些手动的处理，结合ldsc软件的一些名称修改建议，再加上关于一些统计量的转变关系，最终形成了这个python包

- 支持自动识别列名，并进行补充
- 输入文件必须拥有几个必须的列名，`P`,`rsid`,以及`REF A1` 和`ALT A2`,否则怎么都转变不出来合格的格式

## 示例文件

简单做个用法说明和测试，首先已经有`[data/fake_table.GWAS.sumstats.tsv](../data/fake_table.GWAS.sumstats.tsv)`和数据库文件`[data/snp2chrpos.json](../data/snp2chrpos.json)`,`[data/snp2maf.json](../data/snp2maf.json)`。 当然这个三个文件，是我伪造的，
也是为了测试，所以搞得很小的测试文件。

### 先看看示例文件的结构

通过一下展示，可以发现json就是{key:values}这样的格式，而输入的矩阵，也是正常的GWAS sumstats 文件，只是少了很多应有的统计量和信息罢了。

In [1]:
!head ../data/snp2chrpos.json

{
"rs62635278":"1:456",
"rs2691333":"2:3455",
"rs62635282":"3:4556657",
"rs62635284":"5:4546",
"rs77233895":"1:3435",
"rs6682375":"2:1232",
"rs6682385":"22:45546",
"rs11580262":"8:232244",
"rs3982632":"8:354546"


In [2]:
!head ../data/fake_table.GWAS.sumstats.tsv

rsid	A1	A2	N	P	OR
rs62635278	C	A	4	0.7911	0.9818
rs2691333	G	T	5	0.8735	0.9924
rs62635282	G	C	4	0.802	0.9823
rs62635284	A	G	4	0.7956	0.9806
rs77233895	A	C	5	0.8814	0.9937
rs6682375	A	G	4	0.8057	0.9817
rs6682385	G	A	4	0.7992	0.9809
rs11580262	A	G	4	0.7887	0.9784
rs3982632	T	G	4	0.7992	0.9819


### 用GStidy进行处理

首先进入python的界面

In [3]:
from gstidy import gstidy
data = gstidy("../data/fake_table.GWAS.sumstats.tsv")
data

The input data lack of << SE >>, ===>:standard errors
The input data lack of << Z >>, ===>:Z-score (0 --> no effect; above 0 --> A1 is trait/risk increasing)
The input data lack of << BETA >>, ===>:[linear/logistic] regression coefficient (0 --> no effect; above 0 --> A1 is trait/risk increasing)
The input data lack of << INFO >>, ===>:INFO score (imputation quality; higher --> better imputation)
The input data lack of << FRQ >>, ===>:Allele frequency
The input data lack of << PHENOTYPE >>, ===>:The disease PHENOTYPE, If you want add ,need -N parameter


,rsid,A1,A2,N,P,OR,SE,BETA,Z,CHR,BP,MAF
0,rs62635278,C,A,4,0.7911,0.9818,0.069343,-0.018368,-0.264882,chr1,456,0.3
1,rs2691333,G,T,5,0.8735,0.9924,0.047917,-0.007629,-0.159214,chr2,3455,0.4
2,rs62635282,G,C,4,0.8020,0.9823,0.071218,-0.017859,-0.250760,chr3,4556657,0.5
3,rs62635284,A,G,4,0.7956,0.9806,0.075626,-0.019591,-0.259046,chr5,4546,0.4
4,rs77233895,A,C,5,0.8814,0.9937,0.042360,-0.006320,-0.149195,chr1,3435,0.9
5,rs6682375,A,G,4,0.8057,0.9817,0.075086,-0.018470,-0.245977,chr2,1232,0.1
6,rs6682385,G,A,4,0.7992,0.9809,0.075810,-0.019285,-0.254383,chr22,45546,0.2
7,rs11580262,A,G,4,0.7887,0.9784,0.081480,-0.021837,-0.267999,chr8,232244,0.4
8,rs3982632,T,G,4,0.7992,0.9819,0.071804,-0.018266,-0.254383,chr8,354546,0.7
